# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [5]:
data_len = len(data)
print(data_len)

4870


In [6]:
#combine columns of interest
data = pd.concat([data['race'], data['call']], axis=1)
data.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


Question 1 - What test is appropriate for this problem? Does CLT apply?

Two sample z-test is appropriate for this problem since we are comparing means of two different populations. The CLT does apply because the samples are >30 and independent of each other.

Question 2 - What are the null and alternate hypotheses?

H0 : black callbacks = white callbacks
H1 : black callbacks != white callbacks

In [9]:
# Your solution to Q3 here

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

In [12]:
#compute z-test using frequentist statistics
w_prop = np.sum(w.call)/len(w)
b_prop = np.sum(b.call)/len(b)

p_hat = (np.sum(w.call)+np.sum(b.call))/(len(w)+len(b))

z = (w_prop-b_prop)/np.sqrt(p_hat*(1-p_hat)*((1/len(w)+1/len(b))))
p_value = pval = stats.norm.cdf(-z) * 2

print('Z score: {}'.format(z))
print('p value: {}'.format(p_value))

moe = 1.96 * np.sqrt(p_hat * (1 - p_hat) * ((1 / len(w)) + (1 / len(b))))
ci = (w_prop-b_prop) + np.array([-1, 1]) * moe

print('Margin of error:{}'.format(moe))
print('Confidence interval:{}'.format(ci))

Z score: 4.108412152434346
p value: 3.983886837585077e-05
Margin of error:0.015281912310894095
Confidence interval:[0.01675094 0.04731477]


In [17]:
#compute ci and p value using permutation method
w_call = w.call
b_call = b.call

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate([data1, data2])

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_rep_1 = np.empty(size)
    perm_rep_2 = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_rep_1[i] = func(perm_sample_1)
        
        #Compute the test statistic 2
        perm_rep_2[i] = func(np.random.choice(data_1, size=len(data_1)))-func(np.random.choice(data_2, size=len(data_2)))

    return perm_rep_1, perm_rep_2

def frac_data(data1):
    """Compute fraction of white callbacks."""
    frac_1 = np.sum(data1) / len(data1)
    return frac_1

# Draw 10,000 permutation replicates: perm_replicates
perm_rep_1, perm_rep_2 = draw_perm_reps(w_call, b_call, frac_data, 10000)

#compute ci
ci = np.percentile(perm_rep_2, [2.5, 97.5])
print('Confidence Interval = {}'.format(ci))

# Compute p-value: p
p = np.sum(perm_rep_1 >= frac_data(w_call) ) / len(perm_rep_1)

# Print the result
print('p-value ={}'.format(p))

Confidence Interval = [0.01683778 0.04722793]
p-value =0.0


A p value < 0.5 allows us to reject the null hypothesis that callbacks for black-sounding and white-sounding names are equal.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Conclusion

Given the history of our country and the data provided, two hypotheses were listed:

H0 - callbacks for black sounding names is equal to callbacks for white sounding names or

H1 - callbacks for black sounding names is different from callbacks for white sounding names

Through statistical analysis above, we find, from two different methods, a p value of 0.01. This p value is significantly less than 0.5 which allows us to reject the null hypothesis. It seems black sounding names do not get the same callbacks as white sounding names. 

The above analysis does not indicate that race/name is the most important factor in callback success. The analysis only shows that there is a difference in callbacks between white and black names. There are multiple factors that can affect callbacks for interviews.

To amend my analysis I would use the chi-square test.

In [18]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.44902858418937
p-value: 4.997578389963255e-05


The chi square test shows the same p value as the two sample z test. As a result, we can still reject the null hypothesis.